In [127]:
import pandas as pd
import numpy as np

In [128]:
# список товаров, которые мы порекомендуем пользователю в заданном порядке
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров

# юзер 1 купил данные товары
bought_list = [521, 32, 143, 991]

In [129]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [130]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [131]:
hit_rate_at_k(recommended_list, bought_list)

1

In [132]:
recommended_list = [2, 3, 8, 6, 9, 2, 5, 10] 
bought_list = [1, 5, 6]
prices_recommended = [23, 2233, 534, 56, 1, 75, 394, 1674]
prices_bought = [136, 394, 56]

In [133]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
           
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()

In [134]:
print(f'{recall_at_k(recommended_list, bought_list):.3f}')

0.333


In [135]:
print(f'{money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought):.3f}')

0.096


In [136]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [137]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [138]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [139]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [140]:
def map_k(recommended_list, bought_list, k=5):
    
    result = np.mean([ap_k(recommended_list_3_users[i], bought_list_3_users[i], k) for i in range(len(recommended_list_3_users))])
    
    return result

In [141]:
print(f'{map_k(recommended_list, bought_list):.3f}')

0.333


In [142]:
# по желанию
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    
    #dcg
    dcg = 0
    if recommended_list[0] in bought_list:
        dcg = 1
    
    for i in range(1, len(recommended_list)):
        if recommended_list[i] in bought_list:
            dcg += 1/np.log(i+1)
            
    #idcg
    idcg = 1
    for i in range(1, k):
        idcg += 1/np.log(i+1)  
    
    return dcg / idcg

In [143]:
print(f'{ndcg_at_k(recommended_list, bought_list):.3f}')

0.154


In [170]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    ranks = 0
    for i, item_bought in enumerate(recommended_list):
        if np.isin(item_bought, bought_list):
            ranks = 1 / (i+1)
            break
        else:
            ranks = 0
    return ranks

In [171]:
reciprocal_rank(recommended_list, bought_list)

0.25

In [174]:
def mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5):
    
    result = np.mean([reciprocal_rank(recommended_list_3_users[i], bought_list_3_users[i], k) for i in range(len(recommended_list_3_users))])
    
    return result

In [177]:
print(f'{mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users):.3f}')

0.333
